# Andrew Vaccaro
    BS in Software Engineering
    Data engineer at Empiric Health
    Co-organizer of PyData Denver
    andrew.vaccaro@empirichealth.com

## profiling

In [ ]:
%time print('Hello, world.')

In [ ]:
import time

def count():
    time.sleep(1)
    print('Done.')

%time count()
%timeit count()

## some data

In [ ]:
import pandas as pd
pd.read_csv('insurance.csv').head()

## [dtypes](https://docs.scipy.org/doc/numpy/user/basics.types.html)

In [ ]:
df = pd.read_csv('insurance.csv')
df.info(verbose=False, memory_usage='deep')
df.head()

In [ ]:
df['age'] = df['age'].astype('uint8')
df['children'] = df['children'].astype('uint8')
df.info(verbose=False, memory_usage='deep')

In [ ]:
df['bmi'] = df['bmi'].astype('float32')
df['charges'] = df['charges'].astype('float32')
df.info(verbose=False, memory_usage='deep')

In [ ]:
df['smoker'] = df['smoker'].map({'no': 0, 'yes': 1}).astype('uint8')
df.info(verbose=False, memory_usage='deep')

In [ ]:
df.select_dtypes(include=['uint8']).head()

## [categoricals](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html)

In [ ]:
df.select_dtypes(include=['object']).describe()

In [ ]:
df['region'].astype('category').head()

In [ ]:
df['region'].nbytes

In [ ]:
df['region'].astype('category').nbytes

In [ ]:
for c in df.select_dtypes(include=[object]):
    df[c] = df[c].astype('category')
df.info(verbose=False, memory_usage='deep')
df.dtypes

In [ ]:
df['smoker'] = df['smoker'].map({0: 'no', 1: 'yes'}).astype('category')
df.info(verbose=False, memory_usage='deep')

In [ ]:
df.head()

In [ ]:
df.iloc[0]['smoker'] == 'yes'

In [ ]:
df['smoker'].nbytes

In [ ]:
df['smoker'].map({'no': 0, 'yes': 1}).astype('uint8').nbytes

### new categorical values

In [ ]:
df['region'].dtype

In [ ]:
regions = df['region'].copy()
regions[0] = 'east'

In [ ]:
regions = regions.cat.add_categories(['east'])
regions[0] = 'east'
regions.dtype

In [ ]:
regions.apply(lambda s: s.upper()).head()

In [ ]:
df = pd.read_csv('insurance.csv', dtype={
    'age': 'uint8',
    'sex': 'category',
    'bmi': 'float32',
    'children': 'uint8',
    'smoker': 'category',
    'region': 'category',
    'charges': 'float32'
})
df.info(verbose=False, memory_usage='deep')
df.head()

### when not to use categoricals

In [ ]:
import random
import string

def gen_id():
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=4))

def gen_df(n):
    return pd.DataFrame([gen_id() for _ in range(n)], columns=['id'])

df = gen_df(1000)
df.head()

In [ ]:
df = gen_df(100_000)
df.info(verbose=False, memory_usage='deep')
df['id'] = df['id'].astype('category')
df.info(verbose=False, memory_usage='deep')

In [ ]:
df = pd.DataFrame(['Jul 31, 2009', '2010-01-10', None], columns=['dtm'])
df.info(verbose=False, memory_usage='deep')
df['dtm'] = pd.to_datetime(df['dtm'])
df.info(verbose=False, memory_usage='deep')

## looping

In [ ]:
cc_df = pd.read_csv('creditcard_short.csv')
cc_df.info(verbose=False, memory_usage='deep')
cc_df.head()

In [ ]:
def naive(df):
    for i in range(0, len(df)):
        df.loc[i, 'Amount'] = df.loc[i, 'Amount'] * 100
    return df

df = cc_df.copy()
%time df = naive(df)
df['Amount'].sum()

In [ ]:
def naive_assign(df):
    amounts = []
    for i in range(0, len(df)):
        amounts.append(df.iloc[i]['Amount'] * 100)
    df['Amount'] = amounts
    return df

df = cc_df.copy()
%time df = naive_assign(df)
df['Amount'].sum()

In [ ]:
def iterrows(df):
    amounts = []
    for index, row in df.iterrows():
        amounts.append(row['Amount'] * 100)
    df['Amount'] = amounts
    return df
                   
df = cc_df.copy()
%time df = iterrows(df)
df['Amount'].sum()

In [ ]:
def apply(df):
    df['Amount'] = df.apply(lambda r: r['Amount'] * 100, axis=1)
    return df

df = cc_df.copy()
%time df = apply(df)
df['Amount'].sum()

In [ ]:
def apply_col(df):
    df['Amount'] = df['Amount'].apply(lambda a: a * 100)
    return df

df = cc_df.copy()
%time df = apply_col(df)
df['Amount'].sum()

In [ ]:
def vectorize(df):
    df['Amount'] = df['Amount'] * 100
    return df
    
df = cc_df.copy()
%time df = vectorize(df)
df['Amount'].sum()

In [ ]:
def vectorize_numpy(df):
    df['Amount'] = df['Amount'].values * 100
    return df
    
df = cc_df.copy()
%time df = vectorize_numpy(df)
df['Amount'].sum()

## parallelization

In [ ]:
df = gen_df(1_000_000)
%time df['id'].str.contains('C').sum()

In [ ]:
import regex

def search(s):
    return regex.search('(?<!^C)*C(?!^C)*', s) is not None

In [ ]:
%time df['id'].apply(search).sum()

### multiprocessing

In [ ]:
import numpy as np

from multiprocessing import Pool as ProcessPool

def contains(df):
    return df['id'].apply(search).sum()

def parallelize():
    with ProcessPool(processes=4) as pool:
        split = np.array_split(df, 4)
        return sum(pool.map(contains, split))

%time parallelize()

In [ ]:
from multiprocessing.dummy import Pool as ThreadPool

def contains(df):
    return df['id'].apply(search).sum()

def parallelize_threads():
    with ThreadPool(processes=4) as pool:
        split = np.array_split(df, 4)
        return sum(pool.map(contains, split))

%time parallelize_threads()

In [ ]:
from concurrent.futures import ProcessPoolExecutor

def concurrent_futures():
    with ProcessPoolExecutor(max_workers=4) as executor:
        split = np.array_split(df, 4)
        return sum(executor.map(contains, split))

%time concurrent_futures()

### [pandarallel](https://github.com/nalepae/pandarallel/blob/master/pandarallel/dataframe.py#L54)

In [ ]:
from pandarallel import pandarallel
pandarallel.initialize()

%time df['id'].parallel_apply(search).sum()